# Kaggle Insurance Competition

Link: [Insurance Competition](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction)

Goal: we want to predict the **probability** of an insurance claim being made for any given driver given a set of features.

In [1]:
PATH = '/Users/Alex/data/insurance_kaggle/'

Initialisation:

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from fastai.imports import * # fastai.imports imports range of different libraries e.g. pandas
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier # Random Forest Class's we'll use.
from IPython.display import display

from sklearn import metrics

## Data Cleaning:

In [4]:
df_raw = pd.read_csv(f'{PATH}train.csv')

In [5]:
df_raw.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


From the competition information we find:
- Features from similar groupings are tagged as such with calc/ind/reg etc.
- _bin -> binary feature
- _cat -> categorical feature
- Features without suffix bin/cat are either ordinal or numeric

Helpfully, with the numeric codes, this data is already in a good format to feed into the random forest model.

In [6]:
df, y, nas = proc_df(df_raw, 'target')

## Setting up Model Testing:

Create validation set:

In [12]:
set_rf_samples(1000)

In [42]:
reset_rf_samples()

In [13]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy() # Note use .copy() to create a new object in memory

n_valid = 1000  # Specify validation set size
n_trn = len(df)-n_valid # Resulting train set size
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

# Training data: X_train
# Training response: y_train

# Validation set: X_valid

Defining a loss function:

In [14]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean()) # Loss function as per Kaggle

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

Note this isn't exactly how the competition will be judged, but we hope it'll approximate this.

## Building Model:

In [55]:
RF = RandomForestRegressor(n_estimators = 50, n_jobs = -1, min_samples_leaf = 500)
RF.fit(X_train,y_train)
print_score(RF)

[0.18545623290659713, 0.20867699474305432, 0.020229585288606633, 0.0077001154180860176]


In [23]:
RF.predict(X_train)

array([ 0.24511,  0.14714,  0.26983, ...,  0.04026,  0.06469,  0.0565 ])

Okay, this looks really shit, let's actually put it back into Kaggle and see what we get...

In [56]:
test_set = pd.read_csv('/Users/Alex/data/insurance_kaggle/test.csv')

In [32]:
test_set.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,...,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,...,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,...,4,0,0,4,0,1,1,0,0,1


In [57]:
test_preds = RF.predict(test_set)

In [34]:
test_preds

array([ 0.24517,  0.24714,  0.25018, ...,  0.03921,  0.06382,  0.07596])

In [58]:
test_set['target'] = test_preds

In [59]:
submission = test_set[['id','target']]

In [61]:
submission.to_csv('/Users/Alex/Dropbox/submission2.csv', index = False)

Problems I've encountered:
- The r^2 metric is below zero...
- This improves when we put it on Kaggle
- My score actually got worse when I added way more data initially, so tuned the parameters to min_samples_leaf = 500, should hopefully give a more accurate estimate from more averaging

Note that using a RF makes sense here because we average over a number of similar outcomes (0/1) which gives an empirical probability to input.